In [58]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)

# Chemin vers le fichier JSONL
file_path = '/home/carolus/Documents/school/green_ia/data/mini_openfoodfacts.jsonl'

# Lire le fichier JSONL en utilisant la méthode read_json de pandas avec l'option lines=True
df = pd.read_json(file_path, lines=True)

# Afficher le DataFrame
print(df)


    allergens_from_ingredients            pnns_groups_1  \
0                               Milk and dairy products   
1                                               unknown   
2                                               unknown   
3                                               unknown   
4                                               unknown   
..                         ...                      ...   
995                             Milk and dairy products   
996                                       Sugary snacks   
997                                           Beverages   
998                                             unknown   
999                                             unknown   

                                         ecoscore_data  \
0    {'agribalyse': {'co2_packaging': 0.18031662, '...   
1    {'adjustments': {'origins_of_ingredients': {'e...   
2    {'missing': {'packagings': 1, 'origins': 1, 'l...   
3    {'adjustments': {'threatened_species': {'warni...   
4

In [59]:
df.tail(50)

,allergens_from_ingredients,pnns_groups_1,ecoscore_data,ingredients_tags,packaging,product_name,food_groups_tags,ecoscore_tags,categories_tags,ecoscore_score,labels_tags,countries
950,,unknown,"{'missing_key_data': 1, 'missing_agribalyse_ma...","[en:water, en:vegetable-juine, en:fruit-juice,...",None,Wild Berry,[],[unknown],None,NaN,None,en:United States
951,"en:sesame-seeds, en:mustard, en:gluten, en:mil...",unknown,"{'missing': {'packagings': 1, 'agb_category': ...","[en:biscuit, en:sugar, en:added-sugar, en:disa...",None,Baiocchi,[],[unknown],[en:biscuits-with-pistachio],NaN,None,en:Italy
952,,unknown,"{'missing_key_data': 1, 'agribalyse': {'warnin...",None,None,Oldenlandia by ice cool 6s 320ml,[],[unknown],None,NaN,None,en:Singapore
953,"en:soybeans, soy lecithin",unknown,{'agribalyse': {'warning': 'missing_agribalyse...,"[en:e330, en:e332ii, en:e332, en:sodium-citrat...",None,country time lemonade,[],[unknown],None,NaN,None,en:us
954,,unknown,"{'missing_agribalyse_match_warning': 1, 'missi...",None,None,Flohsamen Schalen,[],[unknown],None,NaN,None,en:germany
955,,Milk and dairy products,{'adjustments': {'packaging': {'non_recyclable...,None,,Tilsiter,"[en:milk-and-dairy-products, en:cheese]",[d],"[en:dairies, en:fermented-foods, en:fermented-...",35.0,[],Česko
956,,unknown,"{'missing_key_data': 1, 'missing_agribalyse_ma...",None,None,Barres gourmandes,[],[unknown],None,NaN,"[en:organic, en:eu-organic]",France
957,"en:milk, cream, wheat flour, butter, cream",unknown,{'adjustments': {'origins_of_ingredients': {'e...,"[en:water, en:potato, en:vegetable, en:root-ve...",None,Cream Of Potato Condensed Soup,[],[unknown],None,NaN,None,United States
958,,unknown,{'agribalyse': {'warning': 'missing_agribalyse...,None,None,None,[],[unknown],None,NaN,None,en:Republic of Lithuania
959,,unknown,"{'missing_agribalyse_match_warning': 1, 'statu...",None,None,Riso Nerone,[],[unknown],None,NaN,None,en:it


In [60]:
nan_counts = df.isna().sum()

# Compter les listes vides
empty_list_counts = df.applymap(lambda x: x == []).sum()

# Afficher les résultats
print("nombre de NaN ou None par colonne :")
print(nan_counts)

nombre de NaN ou None par colonne :
allergens_from_ingredients      4
pnns_groups_1                   4
ecoscore_data                   4
ingredients_tags              711
packaging                     796
product_name                   38
food_groups_tags                4
ecoscore_tags                   4
categories_tags               516
ecoscore_score                764
labels_tags                   592
countries                       7
dtype: int64


/tmp/ipykernel_27219/2457842060.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  empty_list_counts = df.applymap(lambda x: x == []).sum()


In [61]:
print("nombre de listes vides par colonne :")
print(empty_list_counts)

nombre de listes vides par colonne :
allergens_from_ingredients      0
pnns_groups_1                   0
ecoscore_data                   0
ingredients_tags                0
packaging                       0
product_name                    0
food_groups_tags              630
ecoscore_tags                   0
categories_tags                35
ecoscore_score                  0
labels_tags                   112
countries                       0
dtype: int64


In [62]:
df['labels_tags'].head(50)

0            [en:organic, en:eu-organic, en:it-bio-006]
1     [en:no-gluten, en:organic, en:eu-organic, en:n...
2                                                  None
3                                                  None
4                                                  None
5                                                  None
6                                                  None
7                                                  None
8                                                    []
9                                                  None
10                                         [en:organic]
11                                                 None
12    [en:french-meat, en:french-poultry, fr:bleu-bl...
13    [en:organic, en:eu-organic, en:es-eco-019-ct, ...
14                            [en:vegetarian, en:vegan]
15                                                   []
16         [en:organic, en:no-gmos, en:non-gmo-project]
17                  [en:french-meat, en:french-p

In [63]:
df['ingredients_tags'].head(50)

0                                                  None
1                                                  None
2                                                  None
3                                                  None
4                                                  None
5                                                  None
6                                                  None
7                                                  None
8                                                  None
9                                                  None
10    [en:green-peas, en:vegetable, en:legume, en:po...
11    [en:peach, en:fruit, en:prunus-species-fruit, ...
12                                                 None
13                                                 None
14    [en:in-bold, en:the-plastic-tray-contains-55-r...
15                                                 None
16    [en:corn, en:cereal, en:sunflower-oil, en:oil-...
17                                              